All code can be found at https://github.com/frankjoshua/demo-ros-cmdvel-jupyter <br>
More examples at http://roboticsascode.com

# First import the ros python libraries

In [3]:
import rospy as rp

In [4]:
import rosnode
rosnode.get_node_names()

['/rosout']

# Create a node in order to send and recieve ROS messages

In [5]:
rp.init_node('jupyter_demo')
rosnode.get_node_names()

['/jupyter_demo', '/rosout']

In [7]:
topicInfo = rosnode.get_node_info_description('/turtlesim')
print(topicInfo)

Node [/turtlesim]
Publications: 
 * /rosout [rosgraph_msgs/Log]
 * /turtle1/color_sensor [turtlesim/Color]
 * /turtle1/pose [turtlesim/Pose]

Subscriptions: 
 * /turtle1/cmd_vel [unknown type]

Services: 
 * /clear
 * /kill
 * /reset
 * /spawn
 * /turtle1/set_pen
 * /turtle1/teleport_absolute
 * /turtle1/teleport_relative
 * /turtlesim/get_loggers
 * /turtlesim/set_logger_level



#### ** Start Turtle sim **
# List available topics

In [8]:
rp.get_published_topics() # Does not show unsubscribed topics

[['/turtle1/color_sensor', 'turtlesim/Color'],
 ['/rosout', 'rosgraph_msgs/Log'],
 ['/rosout_agg', 'rosgraph_msgs/Log'],
 ['/turtle1/pose', 'turtlesim/Pose']]

# Move the turtle by sending messages to the '/turtle1/cmd_vel' topic

In [9]:
import rostopic as rt
topicInfo = rt.get_info_text('/turtle1/cmd_vel')
print(topicInfo)

Type: geometry_msgs/Twist

Publishers: None

Subscribers: 
 * /turtlesim (http://127.0.0.1:38115/)




# The '/turtle1/cmd_vel' topic is a Twist type

In [10]:
topicType = rt.get_topic_type('/turtle1/cmd_vel')
print(topicType)

('geometry_msgs/Twist', '/turtle1/cmd_vel', None)


In [11]:
import rosmsg
msgInfo = rosmsg.get_msg_text(topicType[0])
print(msgInfo)

geometry_msgs/Vector3 linear
  float64 x
  float64 y
  float64 z
geometry_msgs/Vector3 angular
  float64 x
  float64 y
  float64 z



[WARN] [1578027051.561249]: Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


# Twist message consists of 2 Vector3 messages. <br> Each containing 3 floats. X, Y and Z. <br> One for Linear motion and the second for Angular motion.  

# Linear values are meters per second.<br>Angular values are radians per second.

# Create a publisher to send messages to a topic

In [12]:
from geometry_msgs.msg import Twist
publisher = rp.Publisher('turtle1/cmd_vel', Twist, queue_size=0)

# To move the turtle forward send a Twist message with a positive linear x value to the 'turtle/cmd_vel' topic

In [13]:
twistMessage = Twist()
twistMessage.linear.x = 5.0

publisher.publish(twistMessage)

### The turtle will stop moving if no message is received for over one second. This behavior varies from robot to robot.
# Negative values will move the turtle backwards

In [14]:
twistMessage.linear.x = -5.0
publisher.publish(twistMessage)


# Rotate counter clockwise by sending Twist message with angular z set to a positive value

In [15]:
twistMessage.linear.x = 5.0
twistMessage.angular.z = 6.2
publisher.publish(twistMessage)

# Clockwise with negative values

In [16]:
twistMessage.angular.z = -3.0
publisher.publish(twistMessage)

In [17]:
publisher.publish(twistMessage)

# Use standard python code to control the robot

In [18]:
vel_msg = Twist()

for n in range(5):
    vel_msg.linear.x = 5
    vel_msg.angular.z = 0
    publisher.publish(vel_msg)
    rp.sleep(0.5)
    for i in range(15):
        vel_msg.linear.x = 4 + n * n
        vel_msg.angular.z = 100 / (i + 10)
        publisher.publish(vel_msg)
        rp.sleep(0.25)

# **Jupyter ROS** can be used to create widgets to send messages

In [19]:
import jupyros as jr
from geometry_msgs.msg import Twist
jr.publish('/turtle1/cmd_vel', Twist)

### Right click to move the output to a new view

# Echo topic

In [20]:
def callback(msg):
    print(msg)
jr.subscribe('/turtle1/cmd_vel', Twist, callback)

## Connect to ROS services

In [22]:
import rosservice as rs
rs.get_service_list()

['/spawn',
 '/turtlesim/get_loggers',
 '/turtlesim/set_logger_level',
 '/reset',
 '/rosout/get_loggers',
 '/jupyter_demo/get_loggers',
 '/turtle1/teleport_absolute',
 '/clear',
 '/jupyter_demo/set_logger_level',
 '/rosout/set_logger_level',
 '/turtle1/set_pen',
 '/turtle1/teleport_relative',
 '/kill']

## Call the /reset service to clear the Turtle bot

from std_srvs.srv import Empty
reset = rp.ServiceProxy('/reset', Empty)
reset()

## Import IPYWidgets to create reactive HTML widgets

In [70]:
from IPython.core.display import display, HTML
import ipywidgets as iw
iw.HTML('<img src="https://placekitten.com/g/300/150">')

HTML(value='<img src="https://placekitten.com/g/300/150">')

In [43]:
from ipywidgets import IntSlider
slider = IntSlider(value=50)
slider

IntSlider(value=50)

In [71]:
caption = iw.Output()

def printSliderValue(change):
    caption.clear_output()
    with caption:
        print(str(change.new))
    
slider.observe(printSliderValue, names='value')

display(slider)
display(caption)


IntSlider(value=47)

Output()

In [73]:
javascript = '''
<script src="https://cdnjs.cloudflare.com/ajax/libs/nipplejs/0.7.3/nipplejs.min.js"></script>
<script>
    var options = {
        zone: document.getElementById('zone_joystick'),
        color: 'blue',
        mode: 'static',
        position: {top: '50%', left: '10%'}
    };
    var manager = nipplejs.create(options);
    manager.on('move', function(evt, data) {
        debug.innerHTML = JSON.stringify(data.position);
      })
</script>
<div id="zone_joystick" style="height:250px;">
<div id="debug"/>
</div>
'''

display(HTML(javascript))

## Can use tools from robotwebtools.org

In [75]:
from jupyros import ros3d
v = ros3d.Viewer()
rc = ros3d.ROSConnection()
tf_client = ros3d.TFClient(ros=rc, fixed_frame='/slamware_map')
v

Viewer()

In [76]:
laser_view = ros3d.LaserScan(topic="/slamware_ros_sdk_server_node/scan", ros=rc, tf_client=tf_client)
g = ros3d.GridModel()
v.objects = [g, laser_view]

In [77]:
laser_view.point_size = 1
laser_view.static_color = "green"

In [78]:
videoFeed = '<img src="http://192.168.2.84:8090/stream/video.mjpeg" width="60%"" height="60%"/>'
display(HTML(videoFeed))

Launch ROS bridge and tf web publisher
## https://rapyuta-robotics.github.io/zethus/